### VAERS Data Analysis

### SQL queries with Aggregrations 

#### Investigate number of adverse events, deaths and recoveries related to vaccination over time using Adverse_Event table. 

In [18]:
%%bigquery
select ONSET_DATE as Date, COUNT(VAERS_ID) as Cases, COUNTIF(DIED = true) as Deaths, COUNTIF(RECOVD = true) as Recovered
from vaers_modeled.Adverse_Event_Beam_DF
where ONSET_DATE is not null 
group by ONSET_DATE
order by ONSET_DATE desc
limit 10

,Date,Cases,Deaths,Recovered
0,2018-12-31,5,0,1
1,2018-12-30,4,0,1
2,2018-12-29,23,0,4
3,2018-12-28,28,0,8
4,2018-12-27,32,0,15
5,2018-12-26,16,0,8
6,2018-12-25,9,0,4
7,2018-12-24,5,0,3
8,2018-12-23,10,0,5
9,2018-12-22,17,0,9


#### Create database view from aggregrate query above (Date vs Cases, Deaths and Recovered) for Google Data Studio.

In [2]:
%%bigquery
create or replace view vaers_modeled.v_Cases_By_Date as
select ONSET_DATE as Date, COUNT(VAERS_ID) as Cases, COUNTIF(DIED = true) as Deaths, COUNTIF(RECOVD = true) as Recovered
from `studied-brand-266702.vaers_modeled.Adverse_Event_Beam_DF`
where ONSET_DATE is not null 
group by ONSET_DATE
order by ONSET_DATE desc

""


#### View total number of adverse events by state by joining Patient table with Adverse Event table. <br> Notice the highest number of cases happen in California.

In [10]:
%%bigquery
select STATE as State, COUNT(e.VAERS_ID) as Cases
from vaers_modeled.Patient p join vaers_modeled.Adverse_Event_Beam_DF e on p.VAERS_ID = e.VAERS_ID
where STATE is not null and (ONSET_DATE between '2018-1-1' and '2018-12-31')
group by STATE 
order by Cases desc
limit 10

,State,Cases
0,CA,2545
1,TX,1785
2,NY,1552
3,FL,1529
4,PA,1436
5,MI,1196
6,NC,1031
7,OH,994
8,VA,872
9,IL,832


#### Create database view from aggregrate query above (Cases by State) for Google Data Studio.

In [11]:
%%bigquery
create or replace view vaers_modeled.v_Cases_By_State as
select STATE as State, COUNT(e.VAERS_ID) as Cases
from `studied-brand-266702.vaers_modeled.Patient` p join `studied-brand-266702.vaers_modeled.Adverse_Event_Beam_DF` e on p.VAERS_ID = e.VAERS_ID
where STATE is not null and (ONSET_DATE between '2018-1-1' and '2018-12-31')
group by STATE
order by Cases desc

""


#### Join Vaccine table, Vaccination table, Adverse_Event table and Patient table to view number of deaths reported for each vaccine as well as the patient's maximum and minimum age. Filtered to more than 5 deaths and valid patient age. (Age = 0 refer to infants) <br> Pnuemo vaccines are associated with highest number of deaths. These occur to individuals of all ages given the range of patient ages.

In [5]:
%%bigquery
select v.VAX_NAME as Vaccine_Name, COUNT(e.DIED) as Deaths, MAX(AGE_YRS) as Max_Patient_Age, MIN(AGE_YRS) as Min_Patient_Age 
from (vaers_modeled.Vaccine v join vaers_modeled.Vaccination_Beam_DF vax on v.VAX_ID = vax.VAX_ID)
    join vaers_modeled.Adverse_Event_Beam_DF e on vax.VAERS_ID = e.VAERS_ID
    join vaers_modeled.Patient p on vax.VAERS_ID = p.VAERS_ID
where DIED is true 
group by v.VAX_NAME
having Deaths > 5 and Max_Patient_Age is not null
order by Deaths desc

,Vaccine_Name,Deaths,Max_Patient_Age,Min_Patient_Age
0,PNEUMO (PREVNAR13),35,89,0
1,DTAP + HEPB + IPV (PEDIARIX),17,1,0
2,ZOSTER (SHINGRIX),15,87,61
3,HIB (PEDVAXHIB),13,2,0
4,HPV (GARDASIL),12,15,15
5,ROTAVIRUS (ROTATEQ),12,1,0
6,DTAP + IPV + HIB (PENTACEL),11,2,0
7,ROTAVIRUS (ROTARIX),11,1,0
8,HPV (GARDASIL 9),9,40,11
9,INFLUENZA (SEASONAL) (NO BRAND NAME),9,68,47


#### Join Adverse_Event table with Patient table and calculate average age of patients who died by state. <br> Vaccine-related deaths in California (CA) occur to individuals with an average age of 25 years old.  

In [7]:
%%bigquery
select STATE as State, AVG(AGE_YRS) as Average_Age, COUNT(DIED) as Deaths
from vaers_modeled.Patient p join vaers_modeled.Adverse_Event_Beam_DF e on p.VAERS_ID = e.VAERS_ID
where STATE is not null and DIED = true
group by STATE
order by Deaths desc
limit 10

,State,Average_Age,Deaths
0,CA,25.400000,10
1,AZ,25.833333,6
2,WA,13.000000,5
3,NY,46.500000,4
4,TX,25.333333,4
5,KY,20.000000,4
6,NC,17.000000,4
7,OR,59.000000,4
8,FL,40.666667,4
9,IL,45.333333,4


#### Investigate number of deaths associated with the manufacturer of the vaccine connected to the adverse event. <br> Two manufacturers in particular have the highest associated number of deaths.

In [8]:
%%bigquery
select m.VAX_MANU as Vaccine_Manufacturer, COUNT(e.DIED) as Deaths
from (vaers_modeled.Manufacturer m join vaers_modeled.Vaccination_Beam_DF v on v.MANU_ID = m.MANU_ID)
    join vaers_modeled.Adverse_Event_Beam_DF e on v.VAERS_ID = e.VAERS_ID
where m.VAX_MANU != "UNKNOWN MANUFACTURER" and DIED = true
group by m.VAX_MANU
order by Deaths desc

,Vaccine_Manufacturer,Deaths
0,GLAXOSMITHKLINE BIOLOGICALS,80
1,MERCK & CO. INC.,80
2,SANOFI PASTEUR,37
3,PFIZER\WYETH,36
4,"SEQIRUS, INC.",6
5,NOVARTIS VACCINES AND DIAGNOSTICS,3
6,CSL LIMITED,1
7,"MEDIMMUNE VACCINES, INC.",1


#### Join Vaccine table and Manufacturer table through Vaccination table to figure out which vaccine is produced by more than one manufacturer. <br> Only four vaccines are produced by more than one manufacturer.

In [9]:
%%bigquery
select VAX_NAME as Vaccine_Name, COUNT(DISTINCT VAX_MANU) as Num_Manufacturers
from (vaers_modeled.Manufacturer m join vaers_modeled.Vaccination_Beam_DF vax on vax.MANU_ID = m.MANU_ID)
    join vaers_modeled.Vaccine v on v.VAX_ID = vax.VAX_ID
where m.VAX_MANU != "UNKNOWN MANUFACTURER"
group by VAX_NAME
having Num_Manufacturers > 1
order by VAX_NAME

,Vaccine_Name,Num_Manufacturers
0,HEP A (HAVRIX),2
1,INFLUENZA (SEASONAL) (NO BRAND NAME),2
2,"POLIO VIRUS, INACT. (POLIOVAX)",2
3,RABIES (IMOVAX),2


### SQL queries with Subqueries 

#### View states with deaths greater than *average* number of deaths per state, by joining Patient table with Adverse Event table, ordered by total deaths. <br> Notice the highest number of deaths happen in California.

In [11]:
%%bigquery
# List states with deaths greater than average number of deaths per state
select STATE as State, COUNTIF(DIED = true) as Deaths
from vaers_modeled.Patient p join vaers_modeled.Adverse_Event_Beam_DF e on p.VAERS_ID = e.VAERS_ID
where STATE is not null
group by STATE
having Deaths >
# Compute average number of deaths per state
(select AVG(Deaths) as Average_deaths
from
# Compute number of deaths for all states
(select STATE, COUNTIF(DIED = true) as Deaths
from vaers_modeled.Patient p join vaers_modeled.Adverse_Event_Beam_DF e on p.VAERS_ID = e.VAERS_ID
where STATE is not null
group by STATE))
order by Deaths desc
limit 10

,State,Deaths
0,CA,10
1,AZ,6
2,WA,5
3,FL,4
4,TX,4
5,NC,4
6,NY,4
7,IL,4
8,OR,4
9,KY,4


#### List patient ages with highest number of deaths, by joining Patient table with Adverse Event table, ordered by total deaths. <br> Notice that most deaths occur to infants and toddlers.

In [12]:
%%bigquery
# List patient ages with highest number of deaths 
select AGE_YRS as Age, COUNT(AGE_YRS) as Deaths
from
# Retrieve patient ages when they died and that are not null 
(select AGE_YRS 
from vaers_modeled.Patient p join vaers_modeled.Adverse_Event_Beam_DF e on p.VAERS_ID = e.VAERS_ID
where DIED = true and AGE_YRS is not null  and (ONSET_DATE between '2018-1-1' and '2018-12-31'))
group by AGE_YRS
order by Deaths desc
limit 10

,Age,Deaths
0,0,21
1,1,8
2,85,3
3,61,2
4,65,2
5,67,2
6,11,2
7,75,2
8,78,1
9,79,1


#### Create database view from aggregrate query above (Deaths by Age) for Google Data Studio.

In [13]:
%%bigquery
create or replace view vaers_modeled.v_Deaths_By_Age as
select AGE_YRS as Age, COUNT(AGE_YRS) as Deaths
from
(select AGE_YRS 
from `studied-brand-266702.vaers_modeled.Patient` p join `studied-brand-266702.vaers_modeled.Adverse_Event_Beam_DF` e on p.VAERS_ID = e.VAERS_ID
where DIED = true and AGE_YRS is not null and (ONSET_DATE between '2018-1-1' and '2018-12-31'))
group by AGE_YRS
order by Deaths desc

""


#### Find which sex had highest number of hospitilizations by joining Patient and Adverse Event tables. <br> Females have higher hospitilization rates than men.

In [14]:
%%bigquery
# List patient sex with highest number of hospitilizations
select SEX, COUNT(SEX) as Hospitilizations
from
# Retrieve patient sex where hospitilization is true and sex information present.
(select SEX
from vaers_modeled.Patient p join vaers_modeled.Adverse_Event_Beam_DF e on p.VAERS_ID = e.VAERS_ID
where HOSPITAL = true and SEX != "U")
group by SEX
order by Hospitilizations desc

,SEX,Hospitilizations
0,F,1017
1,M,782


#### Find vaccine with highest number of hospitilizations by joining Vaccine and Vaccination tables in order to connect table to Adverse Event via foreign key.

In [15]:
%%bigquery
# List vaccines by number of hospitilizations
select VAX_NAME, COUNT(VAX_NAME) as Hospitilizations
from
# Retrieve number of hospitilizations by vaccine name
(select VAX_NAME
from vaers_modeled.Vaccine v join vaers_modeled.Vaccination_Beam_DF v2 on v.VAX_ID = v2.VAX_ID
join vaers_modeled.Adverse_Event_Beam_DF e on v2.VAERS_ID = e.VAERS_ID
where HOSPITAL = true and VAX_NAME is not null)
group by VAX_NAME
order by Hospitilizations desc
limit 10

,VAX_NAME,Hospitilizations
0,ZOSTER LIVE (ZOSTAVAX),616
1,ZOSTER (SHINGRIX),235
2,PNEUMO (PREVNAR13),199
3,PNEUMO (PNEUMOVAX),167
4,INFLUENZA (SEASONAL) (FLUZONE QUADRIVALENT),92
5,INFLUENZA (SEASONAL) (NO BRAND NAME),91
6,INFLUENZA (SEASONAL) (FLUZONE HIGH-DOSE),87
7,HPV (GARDASIL 9),78
8,ROTAVIRUS (ROTATEQ),78
9,INFLUENZA (SEASONAL) (FLUARIX QUADRIVALENT),77
